In [ ]:
import json
import os
import cv2
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.callbacks import Callback,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
import numpy as np
print(os.listdir('../input'))
%matplotlib inline
#import visualkeras

In [ ]:
!pip install visualkeras
import visualkeras

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
train_df.head()

In [ ]:
train_df['diagnosis'].hist()

In [ ]:
def get_category(level):
    categories = ["Healthy","Mild", "Moderate", "Severe", "Proliferate"]
    return categories[level]

In [ ]:
def show_samples(df):
    fig=plt.figure(figsize=(20, 15))
    for i in range(12):
        image_path = df.loc[i,'id_code']
        image_diagnosis = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig.add_subplot(3, 4, i+1)
        plt.title(get_category(image_diagnosis))
        plt.axis('off')
        plt.imshow(img)
    plt.tight_layout()

show_samples(train_df)

In [ ]:
train_df['diagnosis']=train_df['diagnosis'].astype(str)#for categorical diagnosis
train_df['id_code']=train_df['id_code']+'.png'

In [ ]:
train,test = train_test_split(train_df, test_size=0.20)
print((train.shape, test.shape))

In [ ]:
def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,# randomly flip images
        rotation_range=360,
        rescale=1./255)

In [ ]:
batch_size = 16
data_generator=create_datagen().flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical"
)

In [ ]:
def create_test_datagen():
    return ImageDataGenerator(rescale=1./255)
batch_size = 16
test_generator=create_test_datagen().flow_from_dataframe(
    dataframe=test,
    directory="../input/aptos2019-blindness-detection/train_images",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=batch_size,
    shuffle=True,
    class_mode="categorical"
)

In [ ]:
IMG_SIZE=256

In [ ]:
efficient = EfficientNetB5(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE,IMG_SIZE,3)
)

In [ ]:
def build_model():
    model = Sequential()
    model.add(efficient)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='softmax'))
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.00005),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
stopping=EarlyStopping(monitor='val_loss',patience=5, min_delta=0.005)
from keras.callbacks import ModelCheckpoint
model_save = ModelCheckpoint('./Model.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
history = model.fit_generator(
    data_generator,
    steps_per_epoch=2929// batch_size,
    validation_data=test_generator,
    epochs=12,
    callbacks=[stopping, model_save])

In [ ]:
from tensorflow import keras
model = keras.models.load_model('./Model.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history["loss"]))

plt.figure(figsize = (16, 16))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predict_x = model.predict(test_generator)
predictions = np.argmax(predict_x, axis = 1)
predictions = predictions.reshape(1,-1)[0]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_generator.classes, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_generator.classes, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, predictions, target_names=["C 0", "C 1", "C 2", "C 3", "C 4"]))